In [1]:
from bertopic import BERTopic
from datasets import load_dataset

In [2]:
dataset = load_dataset("mwarchalowski/grants", "no-shorts-no-dups")

In [3]:
from sentence_transformers import SentenceTransformer

In [4]:
embedding_model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/111k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [7]:
splits = dataset["train"].train_test_split(0.1)
splits

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'abstr', 'text_len'],
        num_rows: 114100
    })
    test: Dataset({
        features: ['id', 'title', 'abstr', 'text_len'],
        num_rows: 12678
    })
})

In [ ]:
embeddings = embedding_model.encode(splits["train"]["abstr"], show_progress_bar=True)
embeddings

In [14]:
embeddings.shape

(10, 1024)